In [1]:
import cv2
import PoseModule as pm  # your poseDetector class

def main():
    cap = cv2.VideoCapture(0)
    detector = pm.poseDetector()
    feedback = "Start Plank"
    hold_time = 0
    good_form = False

    while True:
        ret, img = cap.read()
        if not ret:
            break

        img = detector.findPose(img, draw=True)
        lmList = detector.findPosition(img, draw=False)

        if lmList:
            # Keypoints for shoulder, hip, and ankle
            # LEFT_SHOULDER = 11, LEFT_HIP = 23, LEFT_ANKLE = 27
            shoulder_y = lmList[11][2]
            hip_y = lmList[23][2]
            ankle_y = lmList[27][2]

            # Check if hips are aligned with shoulders and ankles
            hip_to_line = abs(hip_y - ((shoulder_y + ankle_y) / 2))

            if hip_to_line < 20:  # Threshold for good alignment
                feedback = "Good Form ✅"
                good_form = True
                hold_time += 1  # increment frames held
            else:
                feedback = "Hips too high/low ⚠️"
                good_form = False
                hold_time = 0

            # Display hold time in seconds (assuming ~30 FPS)
            cv2.putText(img, f"Hold: {hold_time//30} sec", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Display feedback
            cv2.putText(img, feedback, (10, 120), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 255, 0) if good_form else (0, 0, 255), 2)

        cv2.imshow("Plank Form Checker", img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
